# Reviewed/Commented by KL on 6/2/2022

The American Express Default Prediction Kaggle Challenge asks participants to create a model to predict credit card defaults.  This notebook provided by a competition participant does a fantastic job of doing EDA, although with the data anonymized the amount of feature exploration will be limited.

https://www.kaggle.com/code/ambrosm/amex-eda-which-makes-sense

The main takeaways for the competition is that the selection criteria is a combination of ROC and Recall.

Additionally, while in the training dataset of 458k customerIDs, 74% are good customers without a default, and 26% of customers are bad customers that did have a default (defined as not making a payment for 120 days in a row), the good customer were subsampled at a rate of 20.  Therefore, in reality the ratio is 98% good, to 2% bad.

Further, the provided datasets is rather large, at a total of 50.31 gigs.  A few different public competition notebooks provided competitors with more efficient datasets, specifically parquat and feather.  


### Parquat

https://www.kaggle.com/competitions/amex-default-prediction/discussion/327138

### Feather

https://www.kaggle.com/competitions/amex-default-prediction/discussion/327143

The original total dataset was 50.31 gigs.  Parquet was able to reduce it down to 10.41 gbs, and Feather was able to reduce it down to 5.4 gbs.  Additionally, the datatypes for both included reducing numeric columns to float16, and categorical to 'category' which will of course reduce data as the datatypes are more efficient.

Main takeaway here is to remember that you are often given raw datasets and there is often much room for improvement, especially in file size.

The dataset itself was 5.5m rows for training and 11m rows for test.  It appears that each row is a monthly customer statement, but without overlap in the time periods.  The test data is after the train data.  


# Leading Competition notebooks

There are 2 leading competition notebooks publically available as I write this, one based on LGBM, and one based on RNN.  The LGBM notebook is below and will have some of my comments added (denoted with a KL), and the RNN notebook will also have a link to the notebook.  I reviewed the RNN, but I will note provide a notebook with commentary in the interests of time.  

### LGBM Notebook

This is the one that is below.  Here, we can view the notebook fully run.
https://www.kaggle.com/competitions/amex-default-prediction/code?competitionId=35332&sortBy=voteCount

Score LB 0.792

### RNN notebook

https://www.kaggle.com/competitions/amex-default-prediction/discussion/327761

Score LB 0.789

## Comparison 

The RNN notebook requires signficantly more effort for data preparation.  The author of the LGBM author actually provides a great rundown of the difference between data prep for LGBM vs a NN that I will copy below:

----
Preprocessing for LightGBM is much simpler than for neural networks:

Neural networks can't process missing values; LightGBM handles them automatically.

Categorical features need to be one-hot encoded for neural networks; LightGBM handles them automatically.

With neural networks, you need to think about outliers; tree-based algorithms deal with outliers easily.

Neural networks need scaled inputs; tree-based algorithms don't depend on scaling.

----


Additionally, the LGBM actually scores a tad better than the RNN.  Further, while this isn't part of the competition, it is generally easier to provide an explanation of how a tree based model is making it's decisions over a NN, although IT IS possible, just more computationally expensive.

This shows once again why I greatly prefer using LGBM when it makes sense.  Less data prep and trains much faster than XGboost.

#### End of Review section by KL

# LightGBM Quickstart for the *American Express - Default Prediction* competition

This notebook shows how to apply LightGBM to the competition data, and it introduces a space-efficient way of feature engineering.

It is based on the [EDA which makes sense ⭐️⭐️⭐️⭐️⭐️](https://www.kaggle.com/code/ambrosm/amex-eda-which-makes-sense).

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats
import warnings
from colorama import Fore, Back, Style
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay
from lightgbm import LGBMClassifier, log_evaluation

plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

INFERENCE = True # set to False if you only want to cross-validate


In [3]:
#KL competition specific functions to score the results.

def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex',
            amex_metric(y_true, y_pred),
            True)

# Reading and preprocessing the data

We read the data from @munumbutt's [AMEX-Feather-Dataset](https://www.kaggle.com/datasets/munumbutt/amexfeather). Then we create two groups of features:
- Selected features averaged over all statements of a customer
- Selected features taken from the last statement of a customer

The code has been optimized for memory efficiency rather than readability. In particular, `.iloc[mask_array, columns]` needs much less RAM than the groupby construction used in the previous version of the notebook.

Preprocessing for LightGBM is much simpler than for neural networks:
1. Neural networks can't process missing values; LightGBM handles them automatically.
1. Categorical features need to be one-hot encoded for neural networks; LightGBM handles them automatically.
1. With neural networks, you need to think about outliers; tree-based algorithms deal with outliers easily.
1. Neural networks need scaled inputs; tree-based algorithms don't depend on scaling.

In [4]:
#%%time
features_avg = ['B_1', 'B_11', 'B_16', 'B_17', 'B_18', 'B_2', 'B_20',
                'B_28', 'B_3', 'B_4', 'B_5', 'B_7', 'B_9', 'D_112',
                'D_121', 'D_141', 'D_39', 'D_41', 'D_42', 'D_43',
                'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 
                'D_50', 'D_51', 'D_53', 'D_54', 'D_56', 'D_58', 
                'D_59', 'D_60', 'D_91', 'P_2', 'P_3', 'R_1', 'R_2', 
                'R_27', 'R_3', 'R_7', 'S_11', 'S_26', 'S_3', 'S_5']
features_max = ['B_1', 'B_11', 'B_13', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 
                'B_22', 'B_24', 'B_27', 'B_28', 'B_29', 'B_3', 'B_31', 'B_33', 'B_36', 
                'B_4', 'B_42', 'B_5', 'B_7', 'B_9', 'D_102', 'D_103', 'D_105', 'D_109', 
                'D_110', 'D_112', 'D_113', 'D_115', 'D_121', 'D_124', 'D_128', 'D_129', 
                'D_131', 'D_139', 'D_141', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 
                'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_52', 
                'D_53', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_70', 'D_72', 'D_74', 
                'D_75', 'D_79', 'D_81', 'D_83', 'D_84', 'D_88', 'D_89', 'P_2', 'P_3', 
                'R_1', 'R_10', 'R_11', 'R_26', 'R_28', 'R_3', 'R_4', 'R_5', 'R_7', 'R_8', 
                'S_11', 'S_12', 'S_23', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_7', 'S_8', ]
features_last = ['B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_15', 'B_16',
                 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_22', 'B_23',
                 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3',
                 'B_32', 'B_33', 'B_36', 'B_38', 'B_39', 'B_4', 'B_40',
                 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9',
                 'D_102', 'D_103', 'D_105', 'D_106', 'D_107', 'D_109',
                 'D_112', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120',
                 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 
                 'D_129', 'D_132', 'D_133', 'D_135', 'D_136', 'D_137', 
                 'D_140', 'D_141', 'D_143', 'D_145', 'D_39', 'D_41',
                 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48',
                 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55',
                 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63',
                 'D_64', 'D_66', 'D_70', 'D_72', 'D_73', 'D_74', 'D_75',
                 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_82', 'D_83',
                 'D_84', 'D_86', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96',
                 'P_2', 'P_3', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13',
                 'R_14', 'R_15', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 
                 'R_21', 'R_22', 'R_24', 'R_25', 'R_26', 'R_27', 'R_3',
                 'R_4', 'R_5', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12',
                 'S_13', 'S_15', 'S_17', 'S_20', 'S_22', 'S_23', 
                 'S_24', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_6',
                 'S_7', 'S_8', 'S_9']

for i in ['test', 'train'] if INFERENCE else ['train']:
    df = pd.read_feather(f'../input/amexfeather/{i}_data.ftr')
    #KL pop will remove a column and return it as a pandas series object.  Very efficient.
    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    #kl np.roll Roll array elements along a given axis.
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer
    if i == 'train':
        target = df.loc[last, 'target']
        df.drop(columns=['target'], inplace=True)
    #klgc.collect() performs a garbage collection of all generations, reclaims max amount of memory.
    gc.collect()
    print('Read', i)
    #klSome feature creation here, averaging or maxing features from all of the different statements
    df_avg = (df
              .groupby(cid)
              .mean()[features_avg]
              .rename(columns={f: f"{f}_avg" for f in features_avg})
             )
    gc.collect()
    print('Computed avg', i)
    df_max = (df
              .groupby(cid)
              .max()[features_max]
              .rename(columns={f: f"{f}_max" for f in features_max})
             )
    gc.collect()
    print('Computed max', i)
    df = (df.loc[last, features_last]
          .rename(columns={f: f"{f}_last" for f in features_last})
          .set_index(np.asarray(cid[last]))
         )
    gc.collect()
    print('Computed last', i)
    df = pd.concat([df, df_max, df_avg], axis=1)
    if i == 'train': train = df
    else: test = df
    print(f"{i} shape: {df.shape}")
    del df, df_avg, df_max, cid, last

#print('Shapes:', train.shape, target.shape)
print(f"target shape: {target.shape}")

ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

# Cross-validation

We cross-validate with a five-fold StratifiedKFold.

Notice that lightgbm logs the validation score with the competition's scoring function every hundred iterations.

In [ ]:
%%time
# Cross-validation of the classifier

ONLY_FIRST_FOLD = False

features = [f for f in train.columns if f != 'customer_ID' and f != 'target']
#KL the values appear to be tuned, but I don't see any discussion of running a hyperparamter optimization.
#would be a good place to start to optimize.
def my_booster(random_state=1, n_estimators=1150):
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=95,
                          max_bins=511, random_state=random_state)
      
print(f"{len(features)} features")
score_list = []
y_pred_list = []
kf = StratifiedKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
    start_time = datetime.datetime.now()
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = target.iloc[idx_tr]
    y_va = target.iloc[idx_va]
    
    model = my_booster()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model.fit(X_tr, y_tr,
                  eval_set = [(X_tr, y_tr),(X_va, y_va)], 
                  eval_metric=[lgb_amex_metric],
                  callbacks=[log_evaluation(100)])
    y_va_pred = model.predict_proba(X_va)[:,1]
    score = amex_metric(y_va.values, y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f}{Style.RESET_ALL}")
    score_list.append(score)
    
    if INFERENCE:
        y_pred_list.append(model.predict_proba(test[features])[:,1])
        
    if ONLY_FIRST_FOLD: break # we only want the first fold
    
print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:                       {np.mean(score_list):.5f}{Style.RESET_ALL}")

# Prediction histogram

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(y_va_pred[y_va == 0], bins=np.linspace(0, 1, 21),
         alpha=0.5, density=True, label='0')
plt.hist(y_va_pred[y_va == 1], bins=np.linspace(0, 1, 21),
         alpha=0.5, density=True, label='1')
plt.xlabel('y_pred')
plt.ylabel('density')
plt.title('OOF Prediction histogram', color='k')
plt.legend()
plt.show()

# Calibration diagram

The calibration diagram shows how the model predicts the default probability of customers:

In [ ]:
plt.figure(figsize=(12, 4))
CalibrationDisplay.from_predictions(y_va, y_va_pred, n_bins=50, strategy='quantile', ax=plt.gca())
plt.title('Probability calibration')
plt.show()

# Submission

We submit the mean of the five predictions.

In [ ]:
if INFERENCE:
    sub = pd.DataFrame({'customer_ID': test.index,
                        'prediction': np.mean(y_pred_list, axis=0)})
    sub.to_csv('submission.csv', index=False)
    display(sub)